# Optimized Sphere Strings

In [1]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_channel import *
from kraus_channels import *
from quantum_tools import *
from experimental import *
from spam import *
from copy import deepcopy

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Test

## Recover Map

In [2]:
def generate_map_data(channel_target, inputs_map, noise = 0):
    d = channel_target.d
    U_prep, U_basis = inputs_map

    N_map = U_prep.shape[0]
    init = np.zeros((d,d))
    init[0,0] = 1
    init = tf.cast(init, dtype=precision)
    
    state = tf.repeat(tf.expand_dims(init, axis=0), N_map, axis=0)
    state = apply_unitary(state, U_prep)
    state = channel_target.apply_channel(state)
    targets_map = measurement(state, U_basis)
    
    #add noise
    targets_map = add_noise_to_probs(targets_map, noise=noise)
    
    return targets_map


def model_pipeline(channel_target, noise):
    # Make Benchmark
    #################################################################################
    n = 3
    d = 2**n
    
    spam_target = generate_spam_benchmark(n=n, c1=0.8, c2=0.8)
    inputs_spam, targets_spam = generate_spam_data(spam_target, N_spam=None, noise=noise)

    inputs_map, targets_map = generate_map_data(channel_target, spam_target, N_map=2000, noise=noise)
    #################################################################################

    # Fit Models
    #################################################################################
    spam_model = SPAM(init = InitialState(d, c = None),
                      povm = CorruptionMatrix(d, c = None),
                      optimizer = tf.optimizers.Adam(learning_rate=0.01))
    
    spam_model.pretrain(300)

    spam_model.train(inputs = inputs_spam,
                     targets = targets_spam,
                     num_iter = 2000,
                     verbose = False,
                    )

    model = ModelQuantumMap(channel = KrausMap(d = d, 
                                               rank = d**2,
                                               spam = spam_model,
                                              ),
                            loss_function = ProbabilityMSE(),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            logger = Logger(loss_function = ProbabilityMSE(),
                                            loss_function_val = channel_fidelity_loss),
                           )

    model.train(inputs = inputs_map,
                targets = targets_map,
                inputs_val = None,
                targets_val = [channel_target],
                num_iter = 2000,
                N = 500,
                )
    #################################################################################

    return model

In [18]:
n = 2
d = 2**n


np.random.seed(44)
random.seed(44)
tf.random.set_seed(44)

strings = SphereStrings(N=500, n=n)

_, U = strings.generate_circuits()
inputs_map1 = [U[:250], U[250:]]


np.random.seed(44)
random.seed(44)
tf.random.set_seed(44)

strings = SphereStrings(N=500, n=n)
strings.optimize(100)

_, U = strings.generate_circuits()
inputs_map2 = [U[:250], U[250:]]

tf.Tensor(0.2548201279239014, shape=(), dtype=float64)
tf.Tensor(0.2511381007820764, shape=(), dtype=float64)
tf.Tensor(0.25052139746593954, shape=(), dtype=float64)
tf.Tensor(0.2507835342495644, shape=(), dtype=float64)
tf.Tensor(0.25093134038940174, shape=(), dtype=float64)
tf.Tensor(0.25064567172739943, shape=(), dtype=float64)
tf.Tensor(0.2503026723334869, shape=(), dtype=float64)
tf.Tensor(0.2502348114568634, shape=(), dtype=float64)
tf.Tensor(0.2503771003276566, shape=(), dtype=float64)
tf.Tensor(0.2504663255621077, shape=(), dtype=float64)
tf.Tensor(0.25041027804849697, shape=(), dtype=float64)
tf.Tensor(0.25029094498624704, shape=(), dtype=float64)
tf.Tensor(0.25019661088194195, shape=(), dtype=float64)
tf.Tensor(0.250153594691149, shape=(), dtype=float64)
tf.Tensor(0.25014920355008735, shape=(), dtype=float64)
tf.Tensor(0.25016180295053714, shape=(), dtype=float64)
tf.Tensor(0.2501677180448154, shape=(), dtype=float64)
tf.Tensor(0.25015096672183124, shape=(), dtype=float64)
tf

In [14]:
np.random.seed(44)
random.seed(44)
tf.random.set_seed(44)

kraus_target = KrausMap(d, rank = d)
targets_map1 = generate_map_data(kraus_target, inputs_map1)

model = ModelQuantumMap(channel = KrausMap(d=d, 
                                           rank=d,                                                  ),
                               loss_function = KLDiv(),
                               optimizer = tf.optimizers.Adam(learning_rate=0.01),
                               logger = Logger(loss_function = KLDiv(),
                                               loss_function_val = channel_fidelity_loss),
                               )

model.train(inputs = inputs_map1,
            targets = targets_map1,
            inputs_val = None,
            targets_val = [kraus_target],
            num_iter = 1000,
            N = 500,
            )

  0%|          | 0/1000 [00:00<?, ?it/s]

0.10020282849979806 -0.12849206350578316
0.01869152160675234 -0.2939987827303643
0.008239350853806131 -0.41940079834115956
0.004974319856903076 -0.5188277234294822
0.0034173028975407744 -0.5990425412446577
0.0025417045900555245 -0.6640325210724237
0.001965150718862235 -0.7192117686805138
0.0015397444457239025 -0.7689232085294054
0.001199689889444846 -0.8168071478598746
0.0009052959300563088 -0.8632473100733135
0.0006376173902445781 -0.9064760225764354


In [15]:
np.random.seed(44)
random.seed(44)
tf.random.set_seed(44)

kraus_target = KrausMap(d, rank = d)
targets_map2 = generate_map_data(kraus_target, inputs_map2)

model = ModelQuantumMap(channel = KrausMap(d=d, 
                                           rank=d,                                                  ),
                               loss_function = KLDiv(),
                               optimizer = tf.optimizers.Adam(learning_rate=0.01),
                               logger = Logger(loss_function = KLDiv(),
                                               loss_function_val = channel_fidelity_loss),
                               )

model.train(inputs = inputs_map2,
            targets = targets_map2,
            inputs_val = None,
            targets_val = [kraus_target],
            num_iter = 1000,
            N = 500,
            )

  0%|          | 0/1000 [00:00<?, ?it/s]

0.10349319402418837 -0.12849206350578316
0.0196300277831092 -0.2845043680005441
0.009744046181253749 -0.3824015943089767
0.006183670150131536 -0.466749272394607
0.004273807558717721 -0.543212158741927
0.002989543963135837 -0.612017100411117
0.00216420117744309 -0.6702162796461377
0.0016254391900124637 -0.7204275440487976
0.001269964982439832 -0.7640779205057475
0.0010247324540851909 -0.8041759698746347
0.0008356066561202932 -0.8433566730316518


In [19]:
inputs_map1

[<tf.Tensor: shape=(250, 4, 4), dtype=complex128, numpy=
 array([[[ 0.2183+0.0353j,  0.1021-0.2264j, -0.6233+0.0696j,
          -0.1066+0.6962j],
         [-0.0473+0.2439j, -0.1987-0.0971j, -0.0564-0.7021j,
           0.6167+0.1142j],
         [-0.6167+0.1142j, -0.0564+0.7021j, -0.1987+0.0971j,
           0.0473+0.2439j],
         [-0.1066-0.6962j,  0.6233+0.0696j, -0.1021-0.2264j,
           0.2183-0.0353j]],
 
        [[ 0.0698-0.2656j,  0.9145+0.0265j, -0.075 +0.0401j,
          -0.1867-0.2133j],
         [-0.9129-0.0601j,  0.0444+0.2711j,  0.1787+0.22j  ,
           0.0522-0.0672j],
         [-0.0522-0.0672j,  0.1787-0.22j  ,  0.0444-0.2711j,
           0.9129-0.0601j],
         [-0.1867+0.2133j,  0.075 +0.0401j, -0.9145+0.0265j,
           0.0698+0.2656j]],
 
        [[ 0.0138-0.1432j, -0.6265-0.1873j,  0.0062-0.1595j,
          -0.7058-0.1676j],
         [-0.6297-0.1762j,  0.0648-0.1285j, -0.7086-0.1552j,
           0.0636-0.1464j],
         [-0.0636-0.1464j, -0.7086+0.1552j,  0.

In [28]:
print(tf.abs(tf.linalg.trace(inputs_map2[0][0]@tf.linalg.adjoint(inputs_map2[0][2])))**2)

tf.Tensor(0.10064461915433441, shape=(), dtype=float64)
